In [0]:
import dlt
import pyspark.sql.functions as F

In [0]:
@dlt.table(
    name="guns.bronze.info_data_bronze"
)
def ingest_1():
    df = spark.readStream.format("cloudFiles") \
        .option("cloudFiles.format", "csv") \
        .option("cloudFiles.header", "true") \
        .option("cloudFiles.inferSchema", "true") \
        .option("cloudFiles.validateOptions", "false") \
        .load("/Volumes/guns/data/info") 

    df = df.withColumn("event_timestamp", F.current_timestamp())
    
    return df.select([i for i in df.columns if i not in ("year", "month")])

In [0]:
@dlt.table(
    name="guns.silver.info_silver"
)
def transform():
    df = spark.readStream.table("guns.bronze.info_data_bronze")
    df = df.withWatermark(eventTime="event_timestamp", delayThreshold="90 minutes") \
            .groupBy(F.window(timeColumn="event_timestamp", windowDuration="60 minutes"), F.col("age")) \
            .agg(
                F.count("*").alias("total_incidents")
            )
    return df